# OvP project

## Imports and data loading

In [ ]:
# importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import yake
# import rake
# from rake import RAKE
from rake_nltk import Rake
from keybert import KeyBERT
from langdetect import detect
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# import openai
# from keybert.llm import OpenAI
# from keybert import KeyLLM

# nltk.download('stopwords')
# nltk.download('punkt_tab')
nltk.download('punkt')

In [ ]:
# import data
data = pd.read_excel("raw_data.xlsx")

print(data.head(2))

### Exploring data

In [ ]:

print(data.columns)
teacher_list_raw = data['DOCENT_ROL'].tolist()
print(teacher_list_raw[:5])


In [ ]:
teacher_list = [
    item.strip()
    for entry in teacher_list_raw
    for item in entry.split(';')
]
print(teacher_list[:10])

### Choose keyword model by example

In [ ]:
stop_words_dict = {'eng':stopwords.words('english'), 'nl': stopwords.words('dutch')}

In [ ]:
example_text = "Deze cursus gaat over de visuele cultuur van de Nederlandse zeventiende-eeuwse Republiek. De ongekende commerciële groei van de Verenigde Nederlanden kwam voort uit expansiedrang en kolonialisme, wat onder meer resulteerde in een bloeiende kunst- en rariteitenhandel en een explosie van artistieke creativiteit. Deze cursus onderzoekt de impact van de steeds groter wordende wereld op de beeldende kunsten. Welke rol speelden kunstwerken en andere exotische kunstobjecten die in de huiselijke sfeer werden verzameld en gekoesterd? Wat werd er afgebeeld, waarom en voor wie? Wat onthullen schilderijen over de houding van de Nederlanders ten opzichte van de koloniën, van andere Europese landen en van zichzelf? Wat vertellen rariteitenverzamelingen ons over de verhouding tussen het zelf en de wereld, het vertrouwde en het vreemde, het lokale en het internationale? Maar ook zal er stil worden gestaan bij ontdekkingen die op wetenschappelijk gebied worden gedaan en hoe deze al vlug hun weg weten te vinden in de beeldende kunst. [NB: de cursusinhoud is onder voorbehoud en kan wijzigen]"


lang = detect(example_text)

# print(f"Language: {lang} → Stopwords: {stop_words_dict[lang][:5]}")
print(f"Language: {lang}")


In [ ]:
# yake function
def yake_extract_keywords(text, lang='eng'):
    kw_extractor = yake.KeywordExtractor(
                            lan=lang,           # or appropriate language code
                            n=3,                # max n-gram size (try 2-3)
                            dedupLim=0.8,       # make deduplication stricter
                            dedupFunc='levs',   # try 'levs' or 'jaro' instead of default 'seqm'
                            windowsSize=2,      # context window size
                            top=15              # how many keywords you want
                        )
    keywords = kw_extractor.extract_keywords(text)
    return [kw for kw, score in keywords]

yake_kw = yake_extract_keywords(example_text, lang)
print("yake: ", yake_kw)

In [ ]:
def rake_extract_keywords(text, lang='eng'):
    # rake = RAKE()
    rake = Rake(stopwords=stop_words_dict[lang])
    rake.extract_keywords_from_text(text)
    return rake.get_ranked_phrases()

rake_kw = rake_extract_keywords(example_text, lang)
print("rake: ", rake_kw)

In [ ]:
def keybert_extract_keywords(text, lang='eng'):
    kw_model = KeyBERT(model="paraphrase-multilingual-mpnet-base-v2")
    keywords = kw_model.extract_keywords(
                                text,
                                keyphrase_ngram_range=(1,4),
                                stop_words=stop_words_dict[lang],
                                use_maxsum=True,
                                # nr_candidates=20,
                                top_n=20,
                                use_mmr=True,
                                diversity=0.9,
                                highlight=True
    )
    return keywords

keybert_kw = keybert_extract_keywords(example_text, lang)
print("keybert: ", keybert_kw)

In [ ]:
# chatGPT generated

def extract_keywords_advanced(text, top_n=20):

    # model dat NL + EN begrijpt
    kw_model = KeyBERT(model="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

    # tekst splitsen in zinnen
    sentences = sent_tokenize(text)

    all_keywords = []

    for sent in sentences:
        # per zin keywords halen
        kw = kw_model.extract_keywords(
            sent,
            keyphrase_ngram_range=(1, 3),
            stop_words=stop_words_dict[lang],
            use_mmr=True,
            diversity=0.7,
            top_n=5
        )
        all_keywords.extend([k for k, _ in kw])

    # opschonen en unieke termen behouden
    cleaned = []
    for kw in all_keywords:
        kw = kw.lower().strip()
        kw = re.sub(r'[^a-zà-ÿ0-9\s\-]', '', kw)  # speciale tekens eruit
        if len(kw.split()) > 1 and kw not in cleaned:  # liever zinnen dan losse woorden
            cleaned.append(kw)

    # top_n beperken
    cleaned = cleaned[:top_n]

    return {
        "language_detected": lang,
        "keywords": cleaned
    }

# -------------------------------------------------
# 3️⃣ Test
# -------------------------------------------------

results = extract_keywords_advanced(example_text, top_n=20)

print(f"🗣️ Detected language: {results['language_detected']}")
print("🎯 Top keywords:")
for kw in results["keywords"]:
    print("-", kw)


## Get keywords per topic


In [ ]:
course_dict = {}

for ind, row in data.iterrows():
    # print(row)
    # print("done: ",row[0], row[1], row[2])
    course_code, course_name, teachers, keywords = row
    if pd.isna(keywords):
        continue
        
    teachers = teachers.split(";")
    teachers = list(set([' '.join(item.split()[:-1]) for item in teachers]))
    
    # using yake, keywords not great
    keywords_yake = yake_extract_keywords(keywords)
    
    # use keybert, slightly better but not perfect yet
    kw_model = KeyBERT()
    keywords_bert = kw_model.extract_keywords(keywords,
                                              stop_words='english',
                                              use_maxsum=True,
                                              nr_candidates=20,
                                              top_n=20, 
                                              keyphrase_ngram_range=(1, 3),
                                              use_mmr=True,
                                              diversity=0.7)
    # print(keywords)
    
    course_dict[row[0]] = {"course_code": course_code,
                           "course_name": course_name,
                           "teachers": teachers,
                           "keywords_yake": keywords_yake,
                           "keywords_bert": keywords_bert}
    break
    

print("--------------------------------")
print(course_dict)
    # if ind > 3:
    #     break

### Try using LLM API